# Instalação

In [ ]:
import tensorflow as tf
import pkg_resources

required_version = '2.12.0'

# Verificar se o TensorFlow já está instalado e obter a versão
try:
	version = pkg_resources.get_distribution("tensorflow").version
except pkg_resources.DistributionNotFound:
	version = None

# Verificar a versão e instalar se necessário
if version is None or version != required_version:
	print(f"Instalando TensorFlow {required_version}...")
	!pip install tensorflow=={required_version}
else:
	print(f"Versão do TensorFlow {version} já está instalada.")


In [ ]:
import requests
import pandas as pd
import numpy as np


# Dados Treino Epimeteu1 (entrada 1) 

In [ ]:
diaInicio = '20230301'
diaFinal = '20230531'
idGrupo = '15'
idRegra = '137'
idTipo = '5'
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreinoEntradaSaida?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(url)


In [ ]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)
df.head()


In [ ]:
dfTratadoX = df['campoX'].str.split(',').apply(lambda x: [float(val) for val in x])
Entrada1 = np.stack(dfTratadoX.values)
Entrada1.shape


In [ ]:
Y = np.stack(df['campoY'].values)
Saida1 = Y.reshape(-1,1).astype(int)
Saida1.shape


In [ ]:
# Verificar se há valores não numéricos
has_non_numeric = np.isnan(Entrada1).any() or np.isinf(Entrada1).any()
if has_non_numeric:
    print("Entrada1 array contém valores não numéricos.")
else:
    print("Entrada1 contém apenas valores numéricos.")


In [ ]:
# Verificar se há valores não numéricos
has_non_numeric = np.issubdtype(Saida1.dtype, np.number) is False
if has_non_numeric:
    print("Saida1 contém valores não numéricos.")
else:
    print("Saida1 contém apenas valores numéricos.")


# Dados Treino Epimeteu1 (entrada 2) 

In [ ]:
diaInicio = '20230301'
diaFinal = '20230531'
idGrupo = '15'
idRegra = '137'
idTipo = '2'
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreinoEntradaSaida?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(url)


In [ ]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)
df.head()


In [ ]:
dfTratadoX = df['campoX'].str.split(',').apply(lambda x: [float(val) for val in x])
Entrada2 = np.stack(dfTratadoX.values)
Entrada2.shape


In [ ]:
Y = np.stack(df['campoY'].values)
Saida2 = Y.reshape(-1,1).astype(int)
Saida2.shape


In [ ]:
# Verificar se há valores não numéricos
has_non_numeric = np.isnan(Entrada2).any() or np.isinf(Entrada2).any()
if has_non_numeric:
    print("Entrada2 array contém valores não numéricos.")
else:
    print("Entrada2 contém apenas valores numéricos.")


In [ ]:
# Verificar se há valores não numéricos
has_non_numeric = np.issubdtype(Saida1.dtype, np.number) is False
if has_non_numeric:
    print("Saida2 contém valores não numéricos.")
else:
    print("Saida2 contém apenas valores numéricos.")


# Minha Rede

## Estrutra

In [ ]:
entrada_principal = tf.keras.layers.Input(name='entrada_principal',shape=(150,1))
lstm1_principal = tf.keras.layers.LSTM(name='lstm1_principal', units=64, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform(), return_sequences=True) (entrada_principal)
flatten_principal = tf.keras.layers.Flatten(name='flatten_principal') (lstm1_principal)
dense_principal = tf.keras.layers.Dense(name='dense_principal', units=64, activation='relu')(flatten_principal)

entrada_secundaria = tf.keras.layers.Input(name='entrada_secundaria',shape=(600,1))
lstm1_secundaria = tf.keras.layers.LSTM(name='lstm1_secundaria', units=256, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform(), return_sequences=True) (entrada_secundaria)
lstm2_secundaria = tf.keras.layers.LSTM(name='lstm2_secundaria', units=128, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform(), return_sequences=True) (lstm1_secundaria)
lstm3_secundaria = tf.keras.layers.LSTM(name='lstm3_secundaria', units=64, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform(), return_sequences=True) (lstm2_secundaria)
flatten_secundaria = tf.keras.layers.Flatten(name='flatten_secundaria') (lstm3_secundaria)
dense_secundaria = tf.keras.layers.Dense(name='dense_secundaria', units=64, activation='relu')(flatten_secundaria)

concat = tf.keras.layers.Concatenate(name='concat')([dense_principal, dense_secundaria])
output = tf.keras.layers.Dense(name='output',units=1, activation='sigmoid')(concat)

model = tf.keras.Model(inputs=[entrada_principal, entrada_secundaria], outputs=[output])
model.summary()


## Compilacao

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy',  metrics=['accuracy'])


## Fit - Treino

In [ ]:

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)
Ent1X2 = Entrada1.T
Ent1X2 = Ent1X2[np.newaxis,...]
Ent2X2 = Entrada2.T
Ent2X2 = Ent2X2[np.newaxis,...]

history = model.fit(x=[Ent1X2.T,Ent2X2.T],y=Y,epochs=80,verbose=2,callbacks=[callback])


## Salva Rede

In [ ]:
path='pesos/treino_rede1041.h5'
model.save(path)


# Enviando Rede

In [ ]:
!pip install paramiko


In [ ]:
import os
import paramiko


In [ ]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect("191.252.219.136", username="root", password="Digicom2004$")
sftp = ssh.open_sftp()
sftp.put("pesos/treino_rede1041.h5", "/root/executa-rede/redes/treino_rede1041.h5")
sftp.close()
ssh.close()


# Registra Treinamento

In [ ]:
#import requests
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/executouTreinamento'
myobj = {'id': 1041}
x = requests.post(url, json = myobj)
print(x.text)
